# *MUGCPy*

# **Setup**

In [1]:
%%file setupmugcpytest.sh



echo ""
# read -p "Do you want to start the demo [y/n] $cr > " isStart

# if [ "$isStart" == "y" ]; then
mkdir -p ./inputdata
mkdir -p ./outputdata
mkdir -p ./predictionResults
cd inputdata
wget https://files.rcsb.org/download/3cln.pdb
echo "3cln" >> list.txt
cat list.txt
cd ..
echo "Created all needed folders"
# exit 1;
# else
#   echo "Error: okay goodbye"
#   exit 1;
# fi


Writing setupmugcpytest.sh


In [2]:
# ! jupyter notebook --NotebookApp.iopub_data_rate_limit=1000000000

# **Code**

### Import **Libraries**

In [3]:
import os
import os.path
import numpy as np
import subprocess
import math
import time
import pdb
import shutil
# import networkx as nx
import matplotlib.pyplot as plt
# from biopandas.pdb import PandasPdb
import sys
sys.setrecursionlimit(100000)

### **Fileutil**

In [4]:
from shutil import copyfile
from sys import exit
# package: src.fileutil
class Copy(object):
    """ generated source for class Copy """
    inputFilename = None
    outputFilename = None

    def __init__(self, inputFilename, outputFilename):
        """ generated source for method __init__ """
        self.inputFilename = inputFilename
        self.outputFilename = outputFilename

    @classmethod
    def CopyFile(cls):
        """ generated source for method CopyFile """
        source = input(cls.inputFilename)
        target = input(cls.outputFilename)
        # adding exception handling
        try:
            CopyFile(source, target)
        except IOError as e:
            print("Unable to copy file. %s" % e)
            exit(1)
        except:
            print("Unexpected error:", sys.exc_info())
            exit(1)

        print("\nFile copy done!\n")

        while True:
            print("Do you like to print the file ? (y/n): ")
            check = input()
            if check == 'n':
                break
            elif check == 'y':
                file = open(target, "r")
                print("\nHere follows the file content:\n")
                print(file.read())
                file.close()
                print()
                break
            else:
                continue


### **Utils**

In [5]:
class AngleCalculation(object):
    """ generated source for class AngleCalculation """
    # 
    # 	Return: the angle among a, b and c.
    # 	Input: the coodinates (x,y,z) of a, b and c in order.  
    # 	
    @classmethod
    def point3angle(cls, a1, a2, a3, b1, b2, b3, c1, c2, c3):
        """ generated source for method point3angle """
        ag = float()
        ab1 = float()
        ab2 = float()
        ab3 = float()
        cb1 = float()
        cb2 = float()
        cb3 = float()
        ab1 = a1 - b1
        ab2 = a2 - b2
        ab3 = a3 - b3
        cb1 = c1 - b1
        cb2 = c2 - b2
        cb3 = c3 - b3
        ag = acos((ab1 * cb1 + ab2 * cb2 + ab3 * cb3) / (math.sqrt(ab1 * ab1 + ab2 * ab2 + ab3 * ab3) * math.sqrt(cb1 * cb1 + cb2 * cb2 + cb3 * cb3))) * 180 / 3.1416
        return ag

    @classmethod
    def dihedral_angles(cls, a1, a2, a3, b1, b2, b3, c1, c2, c3, d1, d2, d3):
        """ generated source for method dihedral_angles """
        # 
        # 	Return: the dihedral angle between plane formed by x1,x2,x3, and plane formed by x2,x3,x4.
        # 	Input: the coodinates (x,y,z) of a, b and c in order.  
        # 	
        d12 = float()
        d13 = float()
        d14 = float()
        d23 = float()
        d24 = float()
        d34 = float()
        P = float()
        Q = float()
        angle = float()
        distance = Dist
        d12 = distance.distThreeD(a1, a2, a3, b1, b2, b3)
        d13 = distance.distThreeD(a1, a2, a3, c1, c2, c3)
        d14 = distance.distThreeD(a1, a2, a3, d1, d2, d3)
        d23 = distance.distThreeD(b1, b2, b3, c1, c2, c3)
        d24 = distance.distThreeD(b1, b2, b3, d1, d2, d3)
        d34 = distance.distThreeD(c1, c2, c3, d1, d2, d3)
        P = d12 * d12 * (d23 * d23 + d34 * d34 - d24 * d24) + d23 * d23 * (-(d23 * d23) + d34 * d34 + d24 * d24) + d13 * d13 * (d23 * d23 - d34 * d34 + d24 * d24) - 2 * d23 * d23 * d14 * d14
        Q = (d12 + d23 + d13) * (d12 + d23 - d13) * (d12 - d23 + d13) * (-d12 + d23 + d13) * (d23 + d34 + d24) * (d23 + d34 - d24) * (d23 - d34 + d24) * (-d23 + d34 + d24)
        angle = math.acos(P / math.sqrt(Q)) * 180 / 3.1416
        return angle


In [6]:
class CombinationGenerator(object):
    """ generated source for class CombinationGenerator """
    n = int()
    m = int()

    # exhaust m and take n possibility.
    pre = []

    # previous combination.
    def __init__(self, n, m):
        """ generated source for method __init__ """
        self.n = n
        self.m = m

    def arrayCopy(self,src, srcPos, dest, destPos, length):
        for i in range(length):
            dest[i + destPos] = src[i + srcPos]

    # 
    # 	   * generate one combination
    # 	   * if return null,all combination are generated.
    # 	   
    def next(self):
        """ generated source for method next """
        if self.pre == None:
            # taken the first combination.
            self.pre = [None] * self.n
            i = 0
            while len(self.pre):
                self.pre[i] = i
                i += 1
            ret = [None] * self.n
            self.arrayCopy(self.pre, 0, ret, 0, self.n)

            return ret
        ni = self.n - 1
        maxNi = self.m - 1
        while self.pre[ni] + 1 > maxNi:
            # from right to left find all incremental space
            ni -= 1
            maxNi -= 1
            if ni < 0:
                return None
            # nothing find means the combination is exhausted.
        self.pre[ni] += 1
        ni += 1
        while ni < self.n:
            self.pre[ni] = self.pre[ni - 1] + 1
        ret = [None] * n
        System.arraycopy(self.pre, 0, ret, 0, self.n)
        return ret

    def combinations(self):
        """ generated source for method combinations """
        t = 1
        k = self.m
        i = min(self.n, k - self.n)
        l = 1
        while i >= 1:
            t *= k
            t /= l
            l += 1
        return t


In [7]:
class Dist(object):
    """ generated source for class Dist """
    @classmethod
    def distThreeD(cls, x1, y1, z1, x2, y2, z2):
        """ generated source for method distThreeD """
        distance = math.sqrt((x1 - x2) * (x1 - x2) + (y1 - y2) * (y1 - y2) + (z1 - z2) * (z1 - z2))
        return distance


In [8]:
class FileOperation(object):
    """ generated source for class FileOperation """
#     @classmethod
#     def copy(cls, fileDirName, fileDir, fileName):
#         """ generated source for method copy """
#         inputFile = open(fileDirName)
#         outputFile = open(os.path.join(fileDir, fileName))
#         in_ = inputFile
#         out = outputFile
#         c = int()
#         while (c = in_.read()) != -1):
#         in_.close()
#         out.close()

#     @classmethod
#     def changeName(cls, fileDirName, newFileDirName):
#         """ generated source for method changeName """
#         inputFile = File(fileDirName)
#         outputFile = File(newFileDirName)
#         in_ = FileReader(inputFile)
#         out = FileWriter(outputFile)
#         c = int()
#         while (c = in_.read()) != -1:
#         in_.close()
#         out.close()

#     @classmethod
#     def delete(cls, folder):
#         """ generated source for method delete """
#         #  Construct a File object for the backup created by editing
#         #  this source file. The file probably already exists.
#         #  My editor creates backups by putting ~ at the end of the name.
#         inputFolder = File(folder)
#         fileNameArray = []
#         #  Quick, now, delete it immediately:
#         fileNameArray = inputFolder.list_()
#         if fileNameArray != None:
#             i = 0
#             while len(fileNameArray):
#                 fileName = File(folder + File.separator + fileNameArray[i])
#                 fileName.delete()
#                 i += 1

#     @classmethod
#     def getInputStream(cls, fileName):
#         """ generated source for method getInputStream """
#         try:
#             file_ = File(fileName)
#             return FileInputStream(file_)
#         except Exception as e:
#             return None

    @classmethod
    def getEntriesAsList(cls, fileName):
        """ generated source for method getEntriesAsList """
        #  Return list entries. 
        #  An element of that list is one line of the file fileName.
        fil = open(fileName, "r")
        entries = fil.read().split('\n')
        fil.close()
        return entries

    @classmethod
    def saveResults(cls, entryNames, fileName, option):
        """ generated source for method saveResults """
        #  Write list entryNames in file fileName.
        #  An element in entryNames is one line in fileName. 
        if option == "w":
            try:
                with open(fileName, "wb") as f:
                    i = 0
                    while i < len(entryNames):
                        f.write(str(entryNames[i]) + "\n")
                        i += 1
            except Exception as e:
                pass
        if option == "a":
            try:
                with open(fileName, "ab") as f:
                    i = 0
                    while i < len(entryNames):
                        f.write(str(entryNames[i]) + "\n")
                        i += 1
            except Exception as e:
                pass

#     @classmethod
#     @saveResults.register(object, str, str, str)
#     def saveResults_0(cls, entryNames, fileName, option):
#         """ generated source for method saveResults_0 """
#         #  Write list entryNames in file fileName.
#         #  An element in entryNames is one line in fileName. 
#         if option == "w":
#             try:
#                 bw = BufferedWriter(FileWriter(fileName))
#                 bw.write(entryNames + "\n")
#                 bw.close()
#             except Exception as e:
#                 pass
#         if option == "a":
#             try:
#                 bw = BufferedWriter(FileWriter(fileName, True))
#                 bw.write(entryNames + "\n")
#                 bw.close()
#             except Exception as e:
#                 pass

#     @classmethod
#     def windowsCommand(cls, command):
#         """ generated source for method windowsCommand """
#         try:
#             return Runtime.getRuntime().exec_(command)
#         except Exception as e:
#             e.printStackTrace()
#         return None

    @classmethod
    def concatenateFiles(cls, outFileName, fileList):
        """ generated source for method concatenateFiles """
        outFileContent = []
        file_ = []
        i = 0
        while i < len(fileList)-1:
          fileName = str((fileList[i]))
          file_ = cls.getEntriesAsList(fileName)
          print("This is the file being concatenated", fileName)
          outFileContent.append(file_)
          i += 1
        # FileOperation.saveResults(outFileContent, outFileName, "w")
        with open(outFileName, 'w') as outfile:
            for fname in outFileContent:
                with open(fname) as infile:
                    for line in infile:
                        outfile.write(line)


In [9]:
class ParsePDB(object):
    """ generated source for class ParsePDB """
    @classmethod
    def getLines(cls, PDBFile, field, subfield, atom):
        """ generated source for method getLines """
        #  Return a list, the element of which is a line in the file with name PDBFile
        #  that contains the specified atom in a specified field.
        returnList = []
        fileList = []
        line = ""
        i = 0
        if atom != "allatom":
            if (0 == len(atom)) or (len(atom) >= 4):
                print("error! in getLines")
        fileList = FileOperation.getEntriesAsList(PDBFile)
        while i < len(fileList):
            line = str(fileList[i])
            if field == "HETATM" and subfield != "HOH" and line.startswith(field) and line[17:20].strip() in atom:
                returnList.append(line)
            if field == "ATOM" and subfield == "Res" and line.startswith(field) and line[13:17].startswith(atom):
                returnList.append(line)
            if field == "HETATM" and subfield == "HOH" and line.startswith(field) and line[12: 14].startswith(atom) and line[17: 20].startswith(subfield):
                returnList.append(line)
            if subfield == "HET":
                if field == "HETATM" and subfield != "HOH" and line.startswith(field) and line[12:14].startswith(atom) and ("HOH" in line[17: 20]) == False:
                    returnList.append(line)
            if field == "ATOM" and subfield == "ResNoWater" and (line.startswith("ATOM") or line.startswith("HETATM")) and atom == "allatom":
                if ("HOH" in line[17:20]) == False:
                    returnList.append(line)
            if field == "ATOM" and subfield == "Res" and (line.startswith("ATOM") or line.startswith("HETATM")) and atom == "allatom":
                returnList.append(line)
            i += 1
        return returnList

    @classmethod
    def getBondedLines(cls, PDBFile, atom, bondedAtom):
        """ generated source for method getBondedLines """
        returnList = []
        carbonList = []
        oxygenList = []
        fileList = []
        line = ""
        i = 0
        if 0 == len(atom) or len(atom) > 1 or 0 == len(bondedAtom) or len(bondedAtom) > 1:
            print("error! in getBondedLines")
        atom = atom.upper()
        bondedAtom = bondedAtom.upper()
        temp = ""
        fileList = FileOperation.getEntriesAsList(PDBFile)
        while i < len(fileList):
            line = str(fileList[i])
            if ("ATOM" in line[0:4]) and (bondedAtom in line[13]):
                carbonList.append(line)
                if temp == "" and (len(oxygenList)==0) == False:
                    k = 0
                    while k < len(oxygenList):
                        returnList.append(str(oxygenList[k]))
                        k += 1
                # print(oxygenList)
                # oxygenList = []
                temp = line[13:14]
            if ("ATOM" in line[0:4]) and (atom in line[13]):
                oxygenList.append(line)
                if temp == "C" and ((len(carbonList)==0) == False):
                    k = 0
                    while k < len(carbonList):
                        returnList.append(str(carbonList[k]))
                        k += 1
                # print(carbonList)
                # carbonList = []
                temp = line[13:14]
            if (("TER" in line[0:4]) or ("HETATM" in line[0:6]) and (0 != len(temp))):
                k = 0
                while k < len(oxygenList):
                    returnList.append(str(oxygenList[k]))
                    k += 1
                # print(oxygenList)
                # oxygenList = []
                temp = ""
            i += 1
        return returnList


In [10]:
class PutativeTrue(object):
    """ generated source for class PutativeTrue """
    predictionCount = 0
    correctPredictionCount = 0
    correctPredictionPocketCount = 0
    totalPocketCount = 0
    caCoordinate = []
    caCoordinate1 = []

    @classmethod
    def doPrint(self, ocFile, compsub, PDBID, x):
        """ generated source for method doPrint """
        #   Filename: putative_true
        # 		%
        # 		% Abstract: 
        # 		    % Print out predicted calcium location and associated ligand group
        # 		% version 0.3
        # 		% Kun Zhao
        # 		% Version: 0.2
        # 		% Author: Xue Wang
        # 		% Date: 1/25/2008
        # 		% Replace version 0.1
        # 		% Author: Xue Wang 
        # 		% Date: Jun., 2007
        # 		
        fileName = ""
        fileNamePdb = ""
        currDirectory = os.getcwd()
        self.predictionCount += 1
        jj = 0
        lengthCompsub = len(compsub)
        # print(compsub);
        # print("=========================="+len(compsub));
        resultList = []
        resultListPdb = []
        caList = []
        caList = FileOperation.getEntriesAsList(currDirectory + "/outputdata/" + PDBID + "_ca.dat")
        resultLine = ""
        # doubleFormat = DecimalFormat("      ###0.000;      -###0.000")
        # intFormat = DecimalFormat("      ######")
        doubleFormat = float # Decimal.__format__(3)
        intFormat = float # Decimal.__format__(0)
        serialString = ""
        resiNumString = ""
        serialNum = 0
        resiNum = 0
        ocLine = ""
        atomName = ""
        resiName = ""
        caLine = ""
        calciumX = None
        calciumY = None
        calciumZ = None
        fileName = currDirectory + "/predictionResults/" + PDBID + "_site.txt"
        fileNamePdb = currDirectory + "/predictionResults/" + PDBID + "_site.pdb"
        caX = 0.0
        caY = 0.0
        caZ = 0.0
        minDist = 10000.0
        caODist = 0.0
        closeCa = ""
        while jj < len(caList):
            caLine = str(caList[jj])
            caX = float(caLine.split(" ")[5])
            caY = float(caLine.split(" ")[6])
            caZ = float(caLine.split(" ")[7])
            if Dist.distThreeD(caX, caY, caZ, x[0], x[1], x[2]) < minDist:
                minDist = Dist.distThreeD(caX, caY, caZ, x[0], x[1], x[2])
                # count for selectivity curve
                if minDist < 3.5:
                    self.correctPredictionCount += 1
                closeCa = caLine.split(" ")[1]
            if self.caCoordinate1.contains(caLine) == False and Dist.distThreeD(caX, caY, caZ, x[0], x[1], x[2]) < 3.5:
                self.caCoordinate1.append(caLine)
                self.correctPredictionPocketCount += 1
            if self.caCoordinate.contains(caLine) == False:
                self.caCoordinate.append(caLine)
                self.totalPocketCount += 1
            jj += 1
        while jj < lengthCompsub:
            ocLine = str(ocFile[int(str(compsub[jj]))])
            caODist = caODist + Dist.distThreeD(x[0], x[1], x[2], float(ocLine.split(" ")[5]), float(ocLine.split(" ")[6]), float(ocLine.split(" ")[7]))
            jj += 1
        caODist = caODist / lengthCompsub
        while jj < lengthCompsub:
            ocLine = str(ocFile[int(str(compsub[jj]))])
            serialNum = int(ocLine.split(" ")[1])
            resiNum = int(ocLine.split(" ")[4])
            atomName = ocLine.split(" ")[2]
            resiName = ocLine.split(" ")[3]
            serialString = float(serialNum)
            resiNumString = float(resiNum)
            calciumX = float(x[0])
            calciumY = float(x[1])
            calciumZ = float(x[2])
            df = Decimal().__format__()
            if jj == 0:
                resultLine = (">" + str(self.predictionCount) + "    ")[0:4] + serialString[6 - len(serialString): len(serialString)] + " " + (atomName + "     ")[0:4] + " " + resiNumString[4 - len(resiNumString): len(resiNumString)] + " " + (resiName + "    ")[0:4] + calciumX[8 - len(calciumX): len(calciumX)] + calciumY[8 - len(calciumY): len(calciumY)] + calciumZ[8 - len(calciumZ): len(calciumZ)] + " " + (closeCa + "     ")[0: 5] + " " + float(minDist) + " " + float(caODist)
                resultList.append(resultLine)
            else:
                resultLine = "    " + serialString[6 - len(serialString): len(serialString)] + " " + (atomName + "     ")[0:4] + " " + resiNumString[4 - len(resiNumString): len(resiNumString)] + " " + (resiName + "    ")[0:4] + calciumX[8 - len(calciumX): len(calciumX)] + calciumY[8 - len(calciumY): len(calciumY)] + calciumZ[8 - len(calciumZ): len(calciumZ)] + " " + (closeCa + "     ")[0: 5] + " " + float(minDist) + " " + float(caODist)
                resultList.append(resultLine)
            jj += 1
        resultList.append("\n")
        print('waiting for print of site.txt')
        FileOperation.saveResults(resultList, fileName, "w")
        # FileOperation.saveResults(resultList, currDirectory+"/predictionResults/" + "alre_site.txt", "a");
        print('resultLine1: ',resultLine1)
        resultLine1 = "HETATM      CA    CA" + "           " + resultLine[26: 50]
        FileOperation.saveResults(resultLine1, fileNamePdb, "w")

    @classmethod
    def getCorrectPrediction(cls):
        """ generated source for method getCorrectPrediction """
        return self.correctPredictionCount

    @classmethod
    def getPredictionCount(cls):
        """ generated source for method getPredictionCount """
        return self.predictionCount

    @classmethod
    def getCorrectPredictionPocketCount(cls):
        """ generated source for method getCorrectPredictionPocketCount """
        return self.correctPredictionPocketCount

    @classmethod
    def getTotalPocketCount(cls):
        """ generated source for method getTotalPocketCount """
        return self.totalPocketCount


In [11]:
class SideChainCenterOfMass(object):
    """ generated source for class SideChainCenterOfMass """
    @classmethod
    def sideChainCalculation(cls, sideChain):
        """ generated source for method sideChainCalculation """
        oMass = 16
        nMass = 14
        cMass = 12
        sMass = 32
        xNumerator = 0
        xDenominator = 0
        yNumerator = 0
        yDenominator = 0
        zNumerator = 0
        zDenominator = 0
        cCount = 0
        nCount = 0
        oCount = 0
        sCount = 0
        xResult = None
        yResult = None
        zResult = None
        centerOfMass = None
        xCoordinate = [None] * len(sideChain)
        yCoordinate = [None] * len(sideChain)
        zCoordinate = [None] * len(sideChain)
        l = 0
        while l < len(sideChain):
            xCoordinate[l] = float((str(sideChain[l]))[30:38])
            yCoordinate[l] = float((str(sideChain[l]))[39:46])
            zCoordinate[l] = float((str(sideChain[l]))[47:54])
            type_ = (str(sideChain[l]))[13:14]
            if type_ == "C":
                xNumerator = xNumerator + xCoordinate[l] * cMass
                yNumerator = yNumerator + yCoordinate[l] * cMass
                zNumerator = zNumerator + zCoordinate[l] * cMass
                cCount += 1
            if type_ == "O":
                xNumerator = xNumerator + xCoordinate[l] * oMass
                yNumerator = yNumerator + yCoordinate[l] * oMass
                zNumerator = zNumerator + zCoordinate[l] * oMass
                oCount += 1
            if type_ == "N":
                xNumerator = xNumerator + xCoordinate[l] * nMass
                yNumerator = yNumerator + yCoordinate[l] * nMass
                zNumerator = zNumerator + zCoordinate[l] * nMass
                nCount += 1
            if type_ == "S":
                xNumerator = xNumerator + xCoordinate[l] * sMass
                yNumerator = yNumerator + yCoordinate[l] * sMass
                zNumerator = zNumerator + zCoordinate[l] * sMass
                sCount += 1
            l += 1
        xCoord = float((int(round((xNumerator / (cCount * cMass + oCount * oMass + nCount * nMass + sCount * sMass)) * 1000)) / 1000.00))
        yCoord = float((int(round((yNumerator / (cCount * cMass + oCount * oMass + nCount * nMass + sCount * sMass)) * 1000)) / 1000.00))
        zCoord = float((int(round((zNumerator / (cCount * cMass + oCount * oMass + nCount * nMass + sCount * sMass)) * 1000)) / 1000.00))
        #df = DecimalFormat("###0.000")
        xResult = Decimal(xCoord)
        yResult = Decimal(yCoord)
        zResult = Decimal(zCoord)
        R = str(sideChain[0])
        R = R.replace(R[30: 70], "                                        ")
        R = R.replace(R[12: 17], " R   ")
        R = R.replace(R[77: 78], "R")
        R = R.replace(R[38 - 6: 38], str(xResult))
        R = R.replace(R[46 - 6: 46], str(yResult))
        R = R.replace(R[54 - 6: 54], str(zResult))
        centerOfMass = R.__str__()
        return centerOfMass


In [12]:
class MetalBindingMetal(object):
    """ generated source for class MetalBindingMetal """
    @classmethod
    def getMetal(cls, args, metal):
        """ generated source for method getMetal """
        fileList = []
        resultList = []
        caList = []
        allAtom = []
        allAtomNoWater = []
        currDirectory = os.getcwd()
        fileList = FileOperation.getEntriesAsList(args)
        #  a list of PDBIDs   
        # print(args);
        i = 0
        while i < len(fileList)-1:
            PDBID = (str(fileList[i]))[0:4]
            fileName = currDirectory + "/inputdata/" + PDBID + ".pdb"
            caList = ParsePDB.getLines(fileName, "HETATM", metal, metal)
            # print("This is the caList", caList)
            allAtom = ParsePDB.getLines(fileName, "ATOM", "Res", "allatom")
            allAtomNoWater = ParsePDB.getLines(fileName, "ATOM", "ResNoWater", "allatom")
            j = 0
            while j < len(caList):
                try:
                    getString = str(caList[j])
                    # print("This is loop", j , getString)
                    if ('A' == getString[21]) or (str(getString[16])).isspace() == True:
                        resultList.append(getString[0:6] + " " + getString[6:11] + " " + getString[12:16] + " " + " " + " " + getString[17:20] + " " + " " + " " + getString[22:26] + " " + " " + " " + getString[30:54])
                except Exception as e:
                    print("Caught UnsupportedOperationException")
                j += 1
            FileOperation.saveResults(allAtomNoWater, currDirectory + "/outputdata/" + PDBID + "_noWater.dat", "w")
            FileOperation.saveResults(allAtom, currDirectory + "/predictionResults/" + PDBID + "_site.pdb", "w")
            FileOperation.saveResults(resultList, currDirectory + "/outputdata/" + PDBID + "_" + metal.lower() + ".dat", "w")
            print("File: " + currDirectory + "/outputdata/" + PDBID + "_" + metal.lower() + ".dat" + " just written")
            resultList = []
            i += 1


In [13]:
class NMRSeperate(object):
    """ generated source for class NMRSeperate """
    @classmethod
    def nMRSeperate(cls, arg):
        """ generated source for method nMRSeperate """
        fileList = []
        fileList1 = []
        pdbList = []
        NMRresult = []
        line = ""
        nextline = ""
        modelCount = 0
        linecount = 0
        currDirectory = os.getcwd()
        fileList1 = FileOperation.getEntriesAsList(arg)
        #  a list of PDBIDs
        j = 0
        while j < len(fileList1)-1:
            # print(fileList1)
            #  iteratively process PDB files
            PDBID = (str(fileList1[j]))[0:4].lower()
            fileName = currDirectory + "/inputdata/" + PDBID + ".pdb"
            fileList = FileOperation.getEntriesAsList(fileName)
            i = 0
            while i < len(fileList):
                pdbList = []
                if i + 1 < len(fileList):
                    linecount = 0
                    line = str(fileList[i])
                    nextline = str(fileList[i + 1])
                    if line.startswith("MODEL") and (nextline.startswith("ATOM") or nextline.startswith("HETATM")):
                        pdbList.append(line)
                        modelCount += 1
                        while nextline.startswith("ENDMDL") == False:
                            linecount += 1
                            nextline = str(fileList[i + linecount])
                            pdbList.append(nextline)
                        modelName = str(modelCount) + ""
                        if 1 == len(modelName):
                            modelName = "MO0" + str(modelCount)
                        if 2 == len(modelName):
                            modelName = "MO" + str(modelCount)
                        NMRresult.append(str(modelName))
                        FileOperation.saveResults(pdbList, currDirectory + "/inputdata/" + modelName + ".pdb", "w")
                        FileOperation.saveResults(modelName, currDirectory + "/inputdata/" + PDBID + "list.txt", "a")
                        i = i + linecount
                i += 1
            bb = 1
            j += 1
        #  end for
        return NMRresult

    @classmethod
    def nMRmergePDB(cls, PDBIDList, PDBname):
        """ generated source for method nMRmergePDB """
        maxModel = 0
        maxM = 0
        fileList = []
        residueList1 = []
        residueList2 = []
        residueList3 = []
        residueList4 = []
        residueList5 = []
        currDirectory = os.getcwd()
        totalPocket = 0
        totalResi = 0
        k = 0
        while k < len(PDBIDList)-1:
            resultCount = 0
            # FileOperation.saveResults(str(FileOperation.getEntriesAsList(currDirectory+"/inputdata/"+ PDBIDList[k] +".pdb")[0]), currDirectory + "/predictionResults/"+ PDBname+ "_site.pdb", "a");
            # fileList=FileOperation.getEntriesAsList(currDirectory+"/predictionResults/"+ PDBIDList.get(k) +"_site.pdb");
            # FileOperation.saveResults(fileList, currDirectory+ "/predictionResults/"+ PDBname+ "_site.pdb", "a");		
            FileOperation.saveResults("ENDMDL                                                                          ", currDirectory + "/predictionResults/" + PDBname + "_site.pdb", "a")
            residueList1 = FileOperation.getEntriesAsList(currDirectory + "/predictionResults/" + PDBIDList[k] + "_site.txt")
            residueList2.extend(residueList1)
            # count the max model
            it = iter(residueList1)
            while it.next():
                elem = str(it.next())
                if 0 != len(elem) and elem[0:1] == ">":
                    resultCount += 1
                    totalPocket += 1
                if 0 != len(elem):
                    totalResi += 1
            if resultCount > maxModel:
                maxModel = resultCount
                maxM = k
            k += 1
        # first model and max Model
        if maxModel > 0:
            residueList5.extend(FileOperation.getEntriesAsList(currDirectory + "/predictionResults/" + PDBIDList[0] + "_site.txt"))
            residueList5.extend(FileOperation.getEntriesAsList(currDirectory + "/predictionResults/" + PDBIDList[maxM] + "_site.txt"))
        else:
            residueList5.extend(FileOperation.getEntriesAsList(currDirectory + "/predictionResults/" + PDBIDList[0] + "_site.txt"))
        # count for the duplicated residues.
        itern = iter(residueList2)
        while itern.next():
            element = str(itern.next())
            if 0 != len(element):
                residueList3.extend(element[16:25])
        residueList3.sort()
        # remove duplicate entries.
        hashSet = set(residueList3)
        residueList4 = [hashSet]
        residueList4.sort()
        count = 0
        i = 0
        while i < len(residueList4):
            element = str(residueList4[i])
            j = 0
            while j < len(residueList3):
                element1 = str(residueList3[j])
                if element == element1:
                    count += 1
                j += 1
            element2 = element + "  " + str(count) + " " + str(float(round(100 * count / totalPocket)) / 100) + " " + str(float(round(100 * count / totalResi)) / 100)
            residueList4.set(element2)
            count = 0
            i += 1
        # FileOperation.saveResults(residueList4, currDirectory+ "/predictionResults/"+ PDBname+ "_resi.txt", "w");
        # FileOperation.saveResults(residueList5, currDirectory+ "/predictionResults/"+ PDBname+ "_core.txt", "w");


### **Filters**

In [14]:
class CenterOfMassFilter(object):
    """ generated source for class CenterOfMassFilter """
    @classmethod
    def rFilter(cls, p, ocFile, rFile, compsub, allAtomNoWater):
        """ generated source for method rFilter """
        flag = 0
        i = 0
        ii = 0
        j = 0
        k = 0
        resiNum = int()
        comResiNum = int()
        resiNumPre = 0
        hydroPhIndex = 0
        ocLine = None
        atomName = None
        resiName = None
        chainName = None
        comChainName = None
        rLine = None
        atomType = None
        comFile = []
        allAtomFile = []
        # get the comFile which contains the center of mass and main-chain information of binding ligand.
        #  also get the allatom file without water and check hydrophobic index
        while flag == 0 and i < len(compsub):
            # get the binding rFile
            ocLine = str(ocFile[(int(str(compsub[i])))])
            atomName = ocLine.split(" ")[2]
            resiName = ocLine.split(" ")[3]
            resiNum = int(ocLine.split(" ")[4])
            chainName = ocLine.split(" ")[11]
            # remove the binding residue from all atom.
            indexBinding = 0
            while indexBinding < len(allAtomNoWater):
                allLine = str(allAtomNoWater[indexBinding])
                if int(str(allLine[23:26]).strip()) == resiNum and allLine[17:20] == resiName and allLine[77:78] == "C":
                    allAtomNoWater.remove(indexBinding)
                    indexBinding = indexBinding - 1
                    # print(indexBinding)
                indexBinding += 1
            # hydrophobic index
            if resiNum != resiNumPre:
                hydroPhIndex = hydroPhIndex + cls.hydroIndex(resiName)
            resiNumPre = resiNum
            # get comFile
            z = 0
            while z < len(rFile):
                rLine = str(rFile[z])
                comResiNum = int(rLine[23:26].strip())
                comChainName = rLine.split(" ")[4]
                if resiNum == comResiNum and chainName == comChainName:
                    comFile.append(rLine)
                    # indicate binding ligand found
                    k = 1
                if k == 1 and resiNum != comResiNum:
                    k = 0
                    break
                z += 1
            i += 1
        # end while
        if hydroPhIndex > 0:
            flag = 1
        # print(hydroPhIndex);
        atomComName = None
        resiComName = None
        resiComNum = int()
        mainChainCarbonX = float()
        mainChainCarbonY = float()
        mainChainCarbonZ = float()
        mainChainOxygenX = float()
        mainChainOxygenY = float()
        mainChainOxygenZ = float()
        distCaMChainO = 0
        distCaMChainC = float()
        distCaMChainR = 0
        mainChainComX = float()
        mainChainComY = float()
        mainChainComZ = float()
        # mainchain oxygen, carbon and center of mass filter.
        while flag == 0 and j < len(compsub):
            #  binding residue
            ocLine = str(ocFile[(int(str(compsub[j])))])
            atomName = ocLine.split(" ")[2]
            resiName = ocLine.split(" ")[3]
            resiNum = int(ocLine.split(" ")[4])
            # calculate the distance of main chain oxygen and calcium
            if atomName == "O" and flag == 0:
                # get the main-chain carbon atom coordinates.
                mainChainCarbonX = float(ocLine.split(" ")[5])
                mainChainCarbonY = float(ocLine.split(" ")[6])
                mainChainCarbonZ = float(ocLine.split(" ")[7])
                # get the main-chain oxygen atom coordinates.
                while ii < len(comFile):
                    rLine = str(comFile[ii])
                    atomComName = rLine.split(" ")[2]
                    resiComName = rLine.split(" ")[3]
                    resiComNum = int(rLine[23:26].strip())
                    if resiNum == resiComNum and atomComName == "O":
                        mainChainOxygenX = float(rLine.split(" ")[6])
                        mainChainOxygenY = float(rLine.split(" ")[7])
                        mainChainOxygenZ = float(rLine.split(" ")[8])
                        distCaMChainO = Dist.distThreeD(p[0], p[1], p[2], mainChainOxygenX, mainChainOxygenY, mainChainOxygenZ)
                        break
                    ii += 1
                # end for
                distCaMChainC = Dist.distThreeD(p[0], p[1], p[2], mainChainCarbonX, mainChainCarbonY, mainChainCarbonZ)
                if distCaMChainC < distCaMChainO:
                    flag = 1
            # end if and mainchain oxygen filter.
            # center of mass filters and another oxygen position filter.
            if atomName == "O" == False and flag == 0:
                # get the main-chain carbon atom coordinates.
                mainChainCarbonX = float(ocLine.split(" ")[5])
                mainChainCarbonY = float(ocLine.split(" ")[6])
                mainChainCarbonZ = float(ocLine.split(" ")[7])
                while ii < len(allAtomNoWater):
                    rLine = str(allAtomNoWater[ii])
                    atomComName = rLine.split(" ")[2]
                    resiComName = rLine.split(" ")[3]
                    resiComNum = int(rLine.split(" ")[5])
                    if resiNum == resiComNum and atomComName == atomName:
                        mainChainComX = float(rLine.split(" ")[6])
                        mainChainComY = float(rLine.split(" ")[7])
                        mainChainComZ = float(rLine.split(" ")[8])
                        distCaMChainR = Dist.distThreeD(p[0], p[1], p[2], mainChainComX, mainChainComY, mainChainComZ)
                        distCaMChainC = Dist.distThreeD(p[0], p[1], p[2], mainChainCarbonX, mainChainCarbonY, mainChainCarbonZ)
                        if (distCaMChainC < distCaMChainR or distCaMChainC < 2.7) and not resiName == "ASP" and not resiName == "GLU":
                            flag = 1
                        break
                    if resiNum == resiComNum and atomComName == "R":
                        mainChainComX = float(rLine.split(" ")[6])
                        mainChainComY = float(rLine.split(" ")[7])
                        mainChainComZ = float(rLine.split(" ")[8])
                        distCaMChainR = Dist.distThreeD(p[0], p[1], p[2], mainChainComX, mainChainComY, mainChainComZ)
                        distCaMChainC = Dist.distThreeD(p[0], p[1], p[2], mainChainCarbonX, mainChainCarbonY, mainChainCarbonZ)
                        if 4.3 < distCaMChainR:
                            flag = 1
                        break
                    ii += 1
                # end for
            # end if and center of mass filter
            j += 1
        # end while filter
        # check clash.
        if flag == 0:
            l = 0
            while l < len(allAtomNoWater):
                rLine = str(allAtomNoWater[l])
                mainChainComX = float(rLine.split(" ")[6])
                mainChainComY = float(rLine.split(" ")[7])
                mainChainComZ = float(rLine.split(" ")[8])
                distCaMChainC = Dist.distThreeD(p[0], p[1], p[2], mainChainComX, mainChainComY, mainChainComZ)
                atomType = rLine[77:78]
                resiNum = int(rLine[23:26].trim())
                if atomType == "N" and distCaMChainC < 2.55:
                    flag = 1
                    break
                if atomType == "C" and distCaMChainC < 2.7:
                    flag = 1
                    break
                if atomType == "O":
                    if distCaMChainC < 1.6:
                        flag = 1
                        break
                if atomType == "H" and distCaMChainC < 2:
                    flag = 1
                    break
                l += 1
        return flag

    @classmethod
    def hydroIndex(cls, resiName):
        """ generated source for method hydroIndex """
        hydroIndex = 0
        if resiName == "ALA":
            hydroIndex = 0.67
        if resiName == "ARG":
            hydroIndex = -2.1
        if resiName == "ASN":
            hydroIndex = -0.6
        if resiName == "ASP":
            hydroIndex = -1.2
        if resiName == "CYS":
            hydroIndex = 0.38
        if resiName == "GLN":
            hydroIndex = -0.22
        if resiName == "GLU":
            hydroIndex = -0.76
        if resiName == "GLY":
            hydroIndex = 0
        if resiName == "HIS":
            hydroIndex = 0.64
        if resiName == "ILE":
            hydroIndex = 1.9
        if resiName == "LEU":
            hydroIndex = 1.9
        if resiName == "LYS":
            hydroIndex = -0.57
        if resiName == "MET":
            hydroIndex = 2.4
        if resiName == "PHE":
            hydroIndex = 2.3
        if resiName == "PRO":
            hydroIndex = 1.2
        if resiName == "SER":
            hydroIndex = 0.01
        if resiName == "THR":
            hydroIndex = 0.52
        if resiName == "TRP":
            hydroIndex = 2.6
        if resiName == "TYR":
            hydroIndex = 1.6
        if resiName == "VAL":
            hydroIndex = 1.5
        return hydroIndex


In [15]:
class ChargeDistanceFilter(object):
    """ generated source for class ChargeDistanceFilter """
    # filter method
    @classmethod
    def filter_distance(cls, flag, ocFile, filename, compsub, lengthLocat, carbonUpperLimit, carbonLowerLimit, infor1, gridInterval):
        """ generated source for method filter_distance """
        ii = 0
        index = 0
        lengthCompsub = len(compsub)
        xx = 0
        yy = 0
        zz = 0
        x0 = [None] * 3
        max_dist = 0.0
        min_dist = 100.0
        min_ind = 0.0
        max_ind = 0.0
        ratio_f = 0.0
        distC_f = float()
        dist_carbon_oxy = float()
        dist_carbon_ca = float()
        dist1 = float()
        dist2 = float()
        angle1 = float()
        angle2 = float()
        dist_carbon_oxy2 = float()
        dist_carbon_ca2 = float()
        dihedralAngle = 0
        outputArray = [None] * 5
        exitflag = 0.0
        fval = 0.0
        fval2 = 0
        x = [None] * 5
        oxygenX = 0
        oxygenY = 0
        oxygenZ = 0
        carbonX = 0
        carbonY = 0
        carbonZ = 0
        oxygenXPlus = 0
        oxygenYPlus = 0
        oxygenZPlus = 0
        oxygenXMinors = 0
        oxygenYMinors = 0
        oxygenZMinors = 0
        carbonXPlus = 0
        carbonYPlus = 0
        carbonZPlus = 0
        carbonXMinors = 0
        carbonYMinors = 0
        carbonZMinors = 0
        ocLine = ""
        resiName = ""
        resiNamePlus = ""
        ocLinePlus = ""
        ocLineMinors = ""
        atomField = ""
        while ii < len(compsub) and flag == 0:
            ocLine = str(ocFile[int(str(compsub[ii]))])
            oxygenX = float((ocLine.split(" "))[5])
            oxygenY = float((ocLine.split(" "))[6])
            oxygenZ = float((ocLine.split(" "))[7])
            xx = xx + oxygenX
            yy = yy + oxygenY
            zz = zz + oxygenZ
            ii += 1
        x0[0] = xx / lengthCompsub
        x0[1] = yy / lengthCompsub
        x0[2] = zz / lengthCompsub
        outputArray = PositionFun.positionFun(x0, compsub, ocFile, gridInterval)
        x[0] = outputArray[0]
        x[1] = outputArray[1]
        x[2] = outputArray[2]
        x[3] = outputArray[4]
        exitflag = outputArray[3]
        fval = outputArray[4]
        if exitflag < 0.5:
            flag = 1
        while ii < lengthCompsub:
            ocLine = str(ocFile[int(str(compsub[ii]))])
            resiName = (ocLine.split(" "))[3]
            atomField = (ocLine.split(" "))[0]
            carbonX = float((ocLine.split(" "))[8])
            carbonY = float((ocLine.split(" "))[9])
            carbonZ = float((ocLine.split(" "))[10])
            if resiName == "HOH" == False and atomField == "HETATM" == False:
                fval2 = fval2 + Dist.distThreeD(x[0], x[1], x[2], carbonX, carbonY, carbonZ)
                index += 1
            ii += 1
        fval2 = fval2 / index
        if exitflag > 0.5:
            while ii < lengthCompsub:
                ocLine = str(ocFile[int(str(compsub[ii]))])
                oxygenX = float((ocLine.split(" "))[5])
                oxygenY = float((ocLine.split(" "))[6])
                oxygenZ = float((ocLine.split(" "))[7])
                carbonX = float((ocLine.split(" "))[8])
                carbonY = float((ocLine.split(" "))[9])
                carbonZ = float((ocLine.split(" "))[10])
                resiName = (ocLine.split(" "))[3]
                atomField = (ocLine.split(" "))[0]
                if ii == lengthLocat - 1:
                    ocLineMinors = str(ocFile[int(str(compsub[ii - 1]))])
                    oxygenXMinors = float((ocLineMinors.split(" "))[5])
                    oxygenYMinors = float((ocLineMinors.split(" "))[6])
                    oxygenZMinors = float((ocLineMinors.split(" "))[7])
                    carbonXMinors = float((ocLineMinors.split(" "))[8])
                    carbonYMinors = float((ocLineMinors.split(" "))[9])
                    carbonZMinors = float((ocLineMinors.split(" "))[10])
                    resiNamePlus = (ocLine.split(" "))[3]
                elif ii == 0:
                    ocLinePlus = str(ocFile[int(str(compsub[ii + 1]))])
                    oxygenXPlus = float((ocLinePlus.split(" "))[5])
                    oxygenYPlus = float((ocLinePlus.split(" "))[6])
                    oxygenZPlus = float((ocLinePlus.split(" "))[7])
                    carbonXPlus = float((ocLinePlus.split(" "))[8])
                    carbonYPlus = float((ocLinePlus.split(" "))[9])
                    carbonZPlus = float((ocLinePlus.split(" "))[10])
                    resiNamePlus = (ocLinePlus.split(" "))[3]
                if ii > 0 and ii < lengthLocat - 1:
                    ocLinePlus = str(ocFile[int(str(compsub[ii + 1]))])
                    ocLineMinors = str(ocFile[int(str(compsub[ii - 1]))])
                    oxygenXPlus = Decimal((ocLinePlus.split(" "))[5])
                    oxygenYPlus = Decimal((ocLinePlus.split(" "))[6])
                    oxygenZPlus = Decimal((ocLinePlus.split(" "))[7])
                    carbonXPlus = Decimal((ocLinePlus.split(" "))[8])
                    carbonYPlus = Decimal((ocLinePlus.split(" "))[9])
                    carbonZPlus = Decimal((ocLinePlus.split(" "))[10])
                    resiNamePlus = (ocLinePlus.split(" "))[3]
                    oxygenXMinors = Decimal((ocLineMinors.split(" "))[5])
                    oxygenYMinors = Decimal((ocLineMinors.split(" "))[6])
                    oxygenZMinors = Decimal((ocLineMinors.split(" "))[7])
                    carbonXMinors = Decimal((ocLineMinors.split(" "))[8])
                    carbonYMinors = Decimal((ocLineMinors.split(" "))[9])
                    carbonZMinors = Decimal((ocLineMinors.split(" "))[10])
                distC_f = Dist.distThreeD(x[0], x[1], x[2], oxygenX, oxygenY, oxygenZ)
                # 
                if max_dist < distC_f:
                    max_dist = distC_f
                    max_ind = ii
                if min_dist > distC_f:
                    min_dist = distC_f
                    min_ind = ii
                if int(str(compsub[ii])) < infor1 and atomField == "HETATM" == False:
                    # c1-c2 distance measurement.
                    if resiName == "ASP" == False and resiName == "GLU" == False:
                        if AngleCalculation.point3angle(x[0], x[1], x[2], oxygenX, oxygenY, oxygenZ, carbonX, carbonY, carbonZ) < 90:
                            flag = 1
                        dist_carbon_oxy = Dist.distThreeD(x[0], x[1], x[2], oxygenX, oxygenY, oxygenZ)
                        dist_carbon_ca = Dist.distThreeD(x[0], x[1], x[2], carbonX, carbonY, carbonZ)
                        #  could be changed
                        if dist_carbon_ca < dist_carbon_oxy:
                            flag = 1
                    if resiName == "ASP" == True or resiName == "GLU" == True:
                        if (ii == 0 and carbonX != carbonXPlus) or (ii == lengthLocat - 1 and carbonX != carbonXPlus) or (ii > 0 and ii < lengthLocat - 1 and carbonX != carbonXPlus and carbonX != carbonXMinors):
                            # monodenate
                            if AngleCalculation.point3angle(x[0], x[1], x[2], oxygenX, oxygenY, oxygenZ, carbonX, carbonY, carbonZ) < 70:
                                flag = 1
                            dist_carbon_oxy = Dist.distThreeD(x[0], x[1], x[2], oxygenX, oxygenY, oxygenZ)
                            dist_carbon_ca = Dist.distThreeD(x[0], x[1], x[2], carbonX, carbonY, carbonZ)
                    if ii < lengthCompsub - 1 and flag == 0:
                        #  bidenated
                        if resiName == "ASP" == True and resiNamePlus == "ASP" == True and carbonX == carbonXPlus or (resiName == "GLU" == True and resiNamePlus == "GLU" == True and carbonX == carbonXPlus):
                            dist1 = Dist.distThreeD(x[0], x[1], x[2], oxygenX, oxygenY, oxygenZ)
                            dist2 = Dist.distThreeD(x[0], x[1], x[2], oxygenXPlus, oxygenYPlus, oxygenZPlus)
                            dist_carbon_oxy = Dist.distThreeD(x[0], x[1], x[2], oxygenX, oxygenY, oxygenZ)
                            dist_carbon_ca = Dist.distThreeD(x[0], x[1], x[2], carbonX, carbonY, carbonZ)
                            dist_carbon_oxy2 = Dist.distThreeD(x[0], x[1], x[2], oxygenXPlus, oxygenYPlus, oxygenZPlus)
                            dist_carbon_ca2 = Dist.distThreeD(x[0], x[1], x[2], carbonXPlus, carbonYPlus, carbonZPlus)
                            angle1 = AngleCalculation.point3angle(x[0], x[1], x[2], oxygenX, oxygenY, oxygenZ, carbonX, carbonY, carbonZ)
                            dihedralAngle = AngleCalculation.dihedral_angles(carbonX, carbonY, carbonZ, oxygenXPlus, oxygenYPlus, oxygenZPlus, oxygenX, oxygenY, oxygenZ, x[0], x[1], x[2])
                            # 
                            # 									 if (dihedralAngle < 130){
                            # 	                                     flag = 1;
                            # 									 }
                            # 									 
                            #  Ca-C1-C2 angle: bidenate should be a straight line.
                            if (angle1 > 15 and angle1 < 165):
                                flag = 1
                            if (dist_carbon_ca > dist_carbon_oxy):
                                flag = 1
                            #      end;
                        # if
                    # if
                # if
                # ii++;
                ii += 1
            # for
        # if(exitflag)
        # -----------------------------------------------------
        # 
        # 		if (fval > 2.9){   
        # 	          flag = 1;
        # 		}
        # 		if(ratio_f > 0.75) {        
        # 	          flag = 1;
        # 		}
        # 		
        if max_dist > carbonUpperLimit:
            flag = 1
        if min_dist < carbonLowerLimit:
            flag = 1
        x[4] = float((flag))
        return x


In [16]:
class ConsecutiveResiduesFilter(object):
    """ generated source for class ConsecutiveResiduesFilter """
    infor1 = int()
    flag = int()
    compsub = []
    ocFile = []

    def setParameters(self, ocFile_1, compsub_1, infor1_1):
        """ generated source for method setParameters """
        self.ocFile = ocFile_1
        self.compsub = compsub_1
        self.infor1 = infor1_1

    def getFlag(self):
        """ generated source for method getFlag """
        self.flag = self.getflag()
        return self.flag

    def getflag(self):
        """ generated source for method getflag """
        flag = 0
        tempR = -100
        resiNum = int()
        m = int()
        ocLine = ""
        resiName = ""
        while m < len(self.compsub):
            ocLine = str(self.ocFile[int(str(self.compsub[m]))])
            resiName = (ocLine.split(" "))[3]
            resiNum = int((ocLine.split(" "))[4])
            if Integer.parseInt(str(self.compsub[m])) < self.infor1:
                if tempR == resiNum:
                    #  if two consecutive ligand oxygens in compsub come from the same residue, and 
                    if resiName == "ASP" == False and resiName == "GLU" == False and resiName == "ASN" == False and resiName == "THR" == False:
                        #  % if the oxygens are not from ASP nor from GLU 
                        flag = 1
                        break
                tempR = resiNum
            m += 1
        return flag


In [17]:
class PositionFun(object):
    """ generated source for class PositionFun """
    @classmethod
    def positionFun(cls, x0, compsub, ocFile, gridInterval):
        """ generated source for method positionFun """
        # find the calcium center.
        ocFileSize = len(ocFile)
        iii = float()
        jjj = float()
        kkk = float()
        dist_calcium_oxy = float()
        dist_calcium_carbon = float()
        angle = float()
        dist_sum = float()
        bondStrength = float()
        carbonBondStrength = float()
        angle_Ca_O_C = float()
        angle_dihedral = float()
        dist_deviation = float()
        dihedralAngle = float()
        PG = 0
        min_G = 10000
        xStar = 1000
        yStar = 1000
        zStar = 1000
        secondMax = float()
        maxValue = float()
        minValue = float()
        fval = 0
        exitflag = float()
        amount1 = float()
        amount2 = float()
        angle1 = float()
        angle2 = float()
        dist_calcium_oxy1 = float()
        dist_calcium_oxy2 = float()
        gridPoint = [None] * 3
        fX = [None] * 5
        indication_conf = int()
        #  boolean
        count = 0
        jj = int()
        ii = int()
        flag = 0
        lengthCompsub = len(compsub)
        outputArray = [None] * 5
        ocLine = ""
        angleComp = AngleCalculation
        compsubInt = [None] * len(compsub)
        resiNum = [None] * ocFileSize
        oxygenX = [None] * ocFileSize
        oxygenY = [None] * ocFileSize
        oxygenZ = [None] * ocFileSize
        carbonX = [None] * ocFileSize
        carbonY = [None] * ocFileSize
        carbonZ = [None] * ocFileSize
        atomField = [None] * ocFileSize
        resiName = [None] * ocFileSize
        atomName = [None] * ocFileSize
        while ii < len(ocFile):
            ocLine = str(ocFile[ii])
            resiNum[ii] = int(ocLine.split(" ")[4])
            oxygenX[ii] = float(ocLine.split(" ")[5])
            oxygenY[ii] = float(ocLine.split(" ")[6])
            oxygenZ[ii] = float(ocLine.split(" ")[7])
            carbonX[ii] = float(ocLine.split(" ")[8])
            carbonY[ii] = float(ocLine.split(" ")[9])
            carbonZ[ii] = float(ocLine.split(" ")[10])
            atomField[ii] = ocLine.split(" ")[0]
            resiName[ii] = ocLine.split(" ")[3]
            atomName[ii] = ocLine.split(" ")[2]
            ii += 1
        while ii < len(compsub):
            compsubInt[ii] = int(str(compsub[ii]))
            ii += 1
        # /
        while iii <= 1:
            while jjj <= 1:
                while kkk <= 1:
                    gridPoint[0] = x0[0]
                    # + iii;     // grid point: x axis
                    gridPoint[1] = x0[1]
                    # + jjj;     // grid point: y axis
                    gridPoint[2] = x0[2]
                    # + kkk;     // grid point: z axis
                    #  initialization
                    dist_max = [None] * lengthCompsub
                    dist_sum = 0
                    bondStrength = 0
                    dist_deviation = 0
                    carbonBondStrength = 0
                    angle_Ca_O_C = 0
                    angle_dihedral = 0
                    flag = 0
                    indication_conf = 0
                    #  for the whole cluster       
                    count = 0
                    maxValue = -1000
                    minValue = 1000
                    secondMax = -1000
                    ii = 0
                    while ii < lengthCompsub and flag == 0:
                        dist_calcium_oxy = Dist.distThreeD(gridPoint[0], gridPoint[1], gridPoint[2], oxygenX[compsubInt[ii]], oxygenY[compsubInt[ii]], oxygenZ[compsubInt[ii]])
                        # first carbon shell
                        if dist_calcium_oxy > 5.2 or dist_calcium_oxy < 1.74:
                            flag = 1
                        else:
                            dist_max[count] = dist_calcium_oxy
                            dist_sum = dist_sum + dist_calcium_oxy
                            count = count + 1
                            if dist_calcium_oxy > maxValue:
                                maxValue = dist_calcium_oxy
                            if dist_calcium_oxy < minValue:
                                minValue = dist_calcium_oxy
                        ii += 1
                    if flag == 0:
                        while jj < count:
                            if dist_max[jj] != maxValue and dist_max[jj] > 0:
                                if dist_max[jj] > secondMax:
                                    secondMax = dist_max[jj]
                            jj += 1
                    #  for each oxygen atoms   
                    if flag == 0:
                        while ii < lengthCompsub:
                            dist_calcium_oxy = Dist.distThreeD(gridPoint[0], gridPoint[1], gridPoint[2], oxygenX[compsubInt[ii]], oxygenY[compsubInt[ii]], oxygenZ[compsubInt[ii]])
                            #  calicum-oxygen distance
                            #  four cases for atom not from ASP and GLU  
                            if atomField[compsubInt[ii]] == "HETATM" == True and resiName[compsubInt[ii]] == "HOH" == False:
                                bondStrength = bondStrength + 10 * (dist_calcium_oxy - 3.5) * (dist_calcium_oxy - 3.5)
                                angle = angleComp.point3angle(gridPoint[0], gridPoint[1], gridPoint[2], oxygenX[compsubInt[ii]], oxygenY[compsubInt[ii]], oxygenZ[compsubInt[ii]], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                if angle < 140:
                                    angle_Ca_O_C = angle_Ca_O_C + (140 - angle) * (140 - angle) / 400
                            elif resiName[compsubInt[ii]] == "HOH" == True:
                                if dist_calcium_oxy < 3.3:
                                    bondStrength = bondStrength + 10 * (dist_calcium_oxy - 3.5) * (dist_calcium_oxy - 3.5)
                                    if dist_calcium_oxy < 1.3:
                                        bondStrength = bondStrength + 400 * (dist_calcium_oxy - 3.5) * (dist_calcium_oxy - 3.5)
                                else:
                                    indication_conf = 1
                            elif resiName[compsubInt[ii]] == "HOH" == False and atomName[compsubInt[ii]] == "O" == True and atomField[compsubInt[ii]] == "ATOM" == True:
                                dist_calcium_carbon = Dist.distThreeD(gridPoint[0], gridPoint[1], gridPoint[2], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                bondStrength = bondStrength + 10 * (dist_calcium_oxy - 3.5) * (dist_calcium_oxy - 3.5)
                                carbonBondStrength = carbonBondStrength + 10 * (dist_calcium_carbon - 4.3) * (dist_calcium_carbon - 4.3)
                                angle = angleComp.point3angle(gridPoint[0], gridPoint[1], gridPoint[2], oxygenX[compsubInt[ii]], oxygenY[compsubInt[ii]], oxygenZ[compsubInt[ii]], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                if angle < 155:
                                    angle_Ca_O_C = angle_Ca_O_C + (155 - angle) * (155 - angle) / 400
                            elif resiName[compsubInt[ii]] == "ASP" == False and resiName[compsubInt[ii]] == "GLU" == False and resiName[compsubInt[ii]] == "THP" == False and atomName[compsubInt[ii]] == "O" == False:
                                dist_calcium_carbon = Dist.distThreeD(gridPoint[0], gridPoint[1], gridPoint[2], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                bondStrength = bondStrength + 10 * (dist_calcium_oxy - 3.5) * (dist_calcium_oxy - 3.5)
                                carbonBondStrength = carbonBondStrength + 10 * (dist_calcium_carbon - 4.3) * (dist_calcium_carbon - 4.3)
                                angle = angleComp.point3angle(gridPoint[0], gridPoint[1], gridPoint[2], oxygenX[compsubInt[ii]], oxygenY[compsubInt[ii]], oxygenZ[compsubInt[ii]], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                if angle < 140:
                                    angle_Ca_O_C = angle_Ca_O_C + (140 - angle) * (140 - angle) / 400
                            if ii < lengthCompsub - 1:
                                if (resiName[compsubInt[ii]] == "ASP" == True and resiName[compsubInt[ii + 1]] == "ASP" == True and atomName[compsubInt[ii]] == "O" == False and carbonX[compsubInt[ii]] == carbonX[compsubInt[ii + 1]]) or (resiName[compsubInt[ii]] == "GLU" == True and resiName[compsubInt[ii + 1]] == "GLU" == True and atomName[compsubInt[ii]] == "O" == False and carbonX[compsubInt[ii]] == carbonX[compsubInt[ii + 1]]):
                                    dist_calcium_carbon = Dist.distThreeD(gridPoint[0], gridPoint[1], gridPoint[2], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                    dist_calcium_oxy1 = dist_calcium_oxy
                                    dist_calcium_oxy2 = Dist.distThreeD(gridPoint[0], gridPoint[1], gridPoint[2], oxygenX[compsubInt[ii + 1]], oxygenY[compsubInt[ii + 1]], oxygenZ[compsubInt[ii + 1]])
                                    angle1 = angleComp.point3angle(gridPoint[0], gridPoint[1], gridPoint[2], oxygenX[compsubInt[ii]], oxygenY[compsubInt[ii]], oxygenZ[compsubInt[ii]], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                    angle2 = angleComp.point3angle(gridPoint[0], gridPoint[1], gridPoint[2], oxygenX[compsubInt[ii + 1]], oxygenY[compsubInt[ii + 1]], oxygenZ[compsubInt[ii + 1]], carbonX[compsubInt[ii + 1]], carbonY[compsubInt[ii + 1]], carbonZ[compsubInt[ii + 1]])
                                    amount1 = dist_calcium_oxy1 * angle1 / (dist_calcium_oxy2 * angle2)
                                    amount2 = dist_calcium_oxy2 * angle2 / (dist_calcium_oxy1 * angle1)
                                    dihedralAngle = angleComp.dihedral_angles(carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]], oxygenX[compsubInt[ii]], oxygenY[compsubInt[ii]], oxygenZ[compsubInt[ii]], oxygenX[compsubInt[ii + 1]], oxygenY[compsubInt[ii + 1]], oxygenZ[compsubInt[ii + 1]], gridPoint[0], gridPoint[1], gridPoint[2])
                                    if amount1 < 1.3 and amount1 > 0.7 and amount2 < 1.3 and amount2 > 0.7:
                                        bondStrength = bondStrength + 5 * (dist_calcium_oxy1 - 3.6) * (dist_calcium_oxy1 - 3.6)
                                        carbonBondStrength = carbonBondStrength + 5 * (dist_calcium_carbon - 4.4) * (dist_calcium_carbon - 4.4)
                                        angle_Ca_O_C = angle_Ca_O_C + (93 - angle1) * (93 - angle1) / 300
                                        if dihedralAngle < 165:
                                            angle_dihedral = angle_dihedral + (dihedralAngle - 165) * (dihedralAngle - 165) / 100
                                    else:
                                        indication_conf = 1
                                        carbonBondStrength = carbonBondStrength + 5 * (dist_calcium_carbon - 3.5) * (dist_calcium_carbon - 3.5)
                                        if amount1 < 0.7 or amount2 > 1.3:
                                            bondStrength = bondStrength + 5 * (dist_calcium_oxy1 - 3.8) * (dist_calcium_oxy1 - 3.8)
                                            angle_Ca_O_C = angle_Ca_O_C + (55 - angle1) * (55 - angle1) / 600
                                        else:
                                            bondStrength = bondStrength + 5 * (dist_calcium_oxy1 - 3.6) * (dist_calcium_oxy1 - 3.6)
                                            if angle1 < 140:
                                                angle_Ca_O_C = angle_Ca_O_C + (140 - angle1) * (140 - angle1) / 600
                                        if dihedralAngle < 165:
                                            angle_dihedral = angle_dihedral + (dihedralAngle - 165) * (dihedralAngle - 165) / 100
                            if ii > 0:
                                if (resiName[compsubInt[ii]] == "ASP" == True and resiName[compsubInt[ii - 1]] == "ASP" == True and atomName[compsubInt[ii]] == "O" == False and carbonX[compsubInt[ii]] == carbonX[compsubInt[ii - 1]]) or (resiName[compsubInt[ii]] == "GLU" == True and resiName[compsubInt[ii - 1]] == "GLU" == True and atomName[compsubInt[ii]] == "O" == False and carbonX[compsubInt[ii]] == carbonX[compsubInt[ii - 1]]):
                                    dist_calcium_carbon = Dist.distThreeD(gridPoint[0], gridPoint[1], gridPoint[2], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                    dist_calcium_oxy1 = dist_calcium_oxy
                                    dist_calcium_oxy2 = Dist.distThreeD(gridPoint[0], gridPoint[1], gridPoint[2], oxygenX[compsubInt[ii - 1]], oxygenY[compsubInt[ii - 1]], oxygenZ[compsubInt[ii - 1]])
                                    angle1 = angleComp.point3angle(gridPoint[0], gridPoint[1], gridPoint[2], oxygenX[compsubInt[ii]], oxygenY[compsubInt[ii]], oxygenZ[compsubInt[ii]], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                    angle2 = angleComp.point3angle(gridPoint[0], gridPoint[1], gridPoint[2], oxygenX[compsubInt[ii - 1]], oxygenY[compsubInt[ii - 1]], oxygenZ[compsubInt[ii - 1]], carbonX[compsubInt[ii - 1]], carbonY[compsubInt[ii - 1]], carbonZ[compsubInt[ii - 1]])
                                    amount1 = dist_calcium_oxy1 * angle1 / (dist_calcium_oxy2 * angle2)
                                    amount2 = dist_calcium_oxy2 * angle2 / (dist_calcium_oxy1 * angle1)
                                    dihedralAngle = angleComp.dihedral_angles(carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]], oxygenX[compsubInt[ii]], oxygenY[compsubInt[ii]], oxygenZ[compsubInt[ii]], oxygenX[compsubInt[ii - 1]], oxygenY[compsubInt[ii - 1]], oxygenZ[compsubInt[ii - 1]], gridPoint[0], gridPoint[1], gridPoint[2])
                                    if amount1 < 1.3 and amount1 > 0.7 and amount2 < 1.3 and amount2 > 0.7:
                                        bondStrength = bondStrength + 5 * (dist_calcium_oxy1 - 3.5) * (dist_calcium_oxy1 - 3.5)
                                        carbonBondStrength = carbonBondStrength + 5 * (dist_calcium_carbon - 4.4) * (dist_calcium_carbon - 4.4)
                                        angle_Ca_O_C = angle_Ca_O_C + (93 - angle1) * (93 - angle1) / 300
                                        if dihedralAngle < 165:
                                            angle_dihedral = angle_dihedral + (dihedralAngle - 165) * (dihedralAngle - 165) / 100
                                    else:
                                        indication_conf = 1
                                        carbonBondStrength = carbonBondStrength + 5 * (dist_calcium_carbon - 3.5) * (dist_calcium_carbon - 3.5)
                                        if amount1 < 0.7 or amount2 > 1.3:
                                            bondStrength = bondStrength + 5 * (dist_calcium_oxy1 - 3.8) * (dist_calcium_oxy1 - 3.8)
                                            angle_Ca_O_C = angle_Ca_O_C + 5 * (55 - angle1) * (55 - angle1) / 600
                                        else:
                                            bondStrength = bondStrength + 5 * (dist_calcium_oxy1 - 3.4) * (dist_calcium_oxy1 - 3.4)
                                            if angle1 < 140:
                                                angle_Ca_O_C = angle_Ca_O_C + (140 - angle1) * (140 - angle1) / 600
                                        if dihedralAngle < 155:
                                            angle_dihedral = angle_dihedral + (dihedralAngle - 155) * (dihedralAngle - 155) / 400
                            if ii > 0 and ii < lengthCompsub - 1:
                                if (resiName[compsubInt[ii]] == "ASP" == True or resiName[compsubInt[ii]] == "GLU" == True) and atomName[compsubInt[ii]] == "O" == False and carbonX[compsubInt[ii]] != carbonX[compsubInt[ii - 1]] and carbonX[compsubInt[ii]] != carbonX[compsubInt[ii + 1]]:
                                    dist_calcium_carbon = Dist.distThreeD(gridPoint[0], gridPoint[1], gridPoint[2], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                    carbonBondStrength = carbonBondStrength + 10 * (dist_calcium_carbon - 4.3) * (dist_calcium_carbon - 4.3)
                                    bondStrength = bondStrength + 10 * (dist_calcium_oxy - 3.4) * (dist_calcium_oxy - 3.4)
                                    angle = angleComp.point3angle(gridPoint[0], gridPoint[1], gridPoint[2], oxygenX[compsubInt[ii]], oxygenY[compsubInt[ii]], oxygenZ[compsubInt[ii]], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                    if angle < 140:
                                        angle_Ca_O_C = angle_Ca_O_C + (140 - angle) * (140 - angle) / 400
                            if ii == 0:
                                if (resiName[compsubInt[ii]] == "ASP" == True or resiName[compsubInt[ii]] == "GLU" == True) and atomName[compsubInt[ii]] == "O" == False and carbonX[compsubInt[ii]] != carbonX[compsubInt[ii + 1]]:
                                    dist_calcium_carbon = Dist.distThreeD(gridPoint[0], gridPoint[1], gridPoint[2], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                    carbonBondStrength = carbonBondStrength + 10 * (dist_calcium_carbon - 4.3) * (dist_calcium_carbon - 4.3)
                                    bondStrength = bondStrength + 10 * (dist_calcium_oxy - 3.4) * (dist_calcium_oxy - 3.4)
                                    angle = angleComp.point3angle(gridPoint[0], gridPoint[1], gridPoint[2], oxygenX[compsubInt[ii]], oxygenY[compsubInt[ii]], oxygenZ[compsubInt[ii]], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                    if angle < 140:
                                        angle_Ca_O_C = angle_Ca_O_C + (140 - angle) * (140 - angle) / 400
                            if ii == lengthCompsub - 1:
                                if (resiName[compsubInt[ii]] == "ASP" == True or resiName[compsubInt[ii]] == "GLU" == True) and atomName[compsubInt[ii]] == "O" == False and carbonX[compsubInt[ii]] != carbonX[compsubInt[ii - 1]]:
                                    dist_calcium_carbon = Dist.distThreeD(gridPoint[0], gridPoint[1], gridPoint[2], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                    carbonBondStrength = carbonBondStrength + 10 * (dist_calcium_carbon - 4.3) * (dist_calcium_carbon - 4.3)
                                    bondStrength = bondStrength + 10 * (dist_calcium_oxy - 3.4) * (dist_calcium_oxy - 3.4)
                                    angle = angleComp.point3angle(gridPoint[0], gridPoint[1], gridPoint[2], oxygenX[compsubInt[ii]], oxygenY[compsubInt[ii]], oxygenZ[compsubInt[ii]], carbonX[compsubInt[ii]], carbonY[compsubInt[ii]], carbonZ[compsubInt[ii]])
                                    if angle < 140:
                                        angle_Ca_O_C = angle_Ca_O_C + (140 - angle) * (140 - angle) / 400
                            ii += 1
                        if indication_conf == 1:
                            dist_deviation = (count - 1) * 5 * (secondMax - minValue) * (secondMax - minValue)
                            dist_sum = (count - 1) * 10 * pow((dist_sum - maxValue) / (count - 1) - 2.0, 2)
                            PG = (bondStrength + angle_Ca_O_C + angle_dihedral + carbonBondStrength + dist_sum + dist_deviation) / (count - 1)
                        else:
                            dist_deviation = count * 5 * (maxValue - minValue) * (maxValue - minValue)
                            dist_sum = count * 10 * pow(dist_sum / count - 2.0, 2)
                            PG = (bondStrength + angle_Ca_O_C + angle_dihedral + carbonBondStrength + dist_sum + dist_deviation) / count
                        if PG < min_G:
                            min_G = PG
                            xStar = gridPoint[0]
                            yStar = gridPoint[1]
                            zStar = gridPoint[2]
                    kkk = kkk + gridInterval
                jjj = jjj + gridInterval
            iii = iii + gridInterval
        if xStar != 1000 or yStar != 1000 or zStar != 1000:
            fX[0] = xStar
            fX[1] = yStar
            fX[2] = zStar
            exitflag = 1
            fX[3] = exitflag
        while ii < lengthCompsub:
            fval = fval + Dist.distThreeD(fX[0], fX[1], fX[2], oxygenX[compsubInt[ii]], oxygenY[compsubInt[ii]], oxygenZ[compsubInt[ii]])
            ii += 1
        fX[4] = fval / count
        outputArray[0] = fX[0]
        outputArray[1] = fX[1]
        outputArray[2] = fX[2]
        outputArray[3] = fX[3]
        outputArray[4] = fX[4]
        return outputArray

### **MUGC**

In [18]:
class MugControllCarbonShell(object):
    """ generated source for class MugControllCarbonShell """
    infor1 = int()
    infor2 = int()
    predictionCount = 0
    arraySize = 0
    gridInterval = float()
    carbonUpperLimit = float()
    carbonLowerLimit = float()
    maxClique = []
    count_maxClique = 0

    #  
    # 	%
    # 	% Abstract: 
    # 	    % Predict Calcium location and calcim-binding ligands
    # 	    % Return tp_ind_array. If a clique of size >= 4 is a true prediction,
    # 	    % the corresponding element in tp_ind_array is the index of the site that it
    # 	    % predicts; if that clique is not a true prediction (either not putative or not true), the
    # 	    % corresponding element is 0. 
    # 	    % Find all the maximal cliques of size >= 4, where C_m is the5 adjacency
    # 	    % matrix.  
    # 	    % Call function "judgeA" to evaluate the found maximal cliques.A found clique is a 
    # 	    % potential validated predicted Ca-binding site.  
    # 	
    def Predict(self, args):
        """ generated source for method Predict """
        fileName = args[0]
        self.gridInterval = float(4)
        self.carbonLowerLimit = float(1.74)
        self.carbonUpperLimit = float(4.9)
        inforFile = []
        currDirectory = os.getcwd()
        inforFile = FileOperation.getEntriesAsList(currDirectory + "/outputdata/" + fileName + "_info1.dat")
        ocFile = []
        ocFile = FileOperation.getEntriesAsList(currDirectory + "/outputdata/" + fileName + "_f1.dat")
        self.arraySize = len(ocFile) + 10
        mFile = [None] * self.arraySize
        mFile = FileOperation.getEntriesAsList(currDirectory + "/outputdata/" + fileName + "_m.dat")
        rFile = []
        rFile = FileOperation.getEntriesAsList(currDirectory + "/outputdata/" + fileName + "_COM.pdb")
        allAtomNoWater = []
        allAtomNoWater = FileOperation.getEntriesAsList(currDirectory + "/outputdata/" + fileName + "_noWater.dat")
        inforString1 = ((str(inforFile[0])).split(" "))[0]
        inforString2 = ((str(inforFile[0])).split(" "))[1]
        self.infor1 = int(inforString1)
        self.infor2 = int(inforString2)
        indicationChar = []
        matrixRow = [None] * len(ocFile)
        compsub = []
        not_ = []
        # candidate is the index of oxygen atoms in a cluster which has >=4 
        candidate = []
        compsub1 = []
        compsub2 = []
        # elasped time of finding maximum clique
        # start = System.currentTimeMillis()
        for ii in range(len(mFile[:-1])):
          sum = 0
          indicationChar = str(mFile[ii]).split(" ")
          for jj in range(len(mFile[:-1])):
            matrixRow[jj] = int((indicationChar[jj]))
            sum = sum + matrixRow[jj];
          if (sum >= 4):
            candidate.append(str(ii));
        # find maximum clique, and set it in the map of maxClique.
        self.maxClique.append(self.fmxlcA(ocFile, compsub, candidate, not_, fileName, len(mFile[:-1])))
        #  Call fmxlcA
        FileOperation.saveResults("There are in total: " + str(len(self.maxClique)) + " maximum cliques found.", currDirectory + "/predictionResults/results.dat", "a")
        i = 0
        while i < len(self.maxClique):
            maximumClique = []
            maximumClique = self.maxClique[i]
            # print(maximumClique.__str__());
            i += 1
        #  Get elapsed time in milliseconds
        # elapsedTimeMillis = System.currentTimeMillis() - start
        # #  Get elapsed time in seconds
        # elapsedTimeSec = elapsedTimeMillis / 1000
        # # print(elapsedTimeSec);
        # start = System.currentTimeMillis()
        # judge all possible clusters and print results.
        i = 0
        while i < len(self.maxClique):
            maximumClique = []
            maximumClique = self.maxClique[i]
            #  compsub contains the index of oxygens in a maximal clique of size >= 4.
            # Iteratively check if each possible clusters is the potential binding sites or not.
            clusterSize = 4
            while clusterSize <= maximumClique:
                # combination generator.
                cG = CombinationGenerator(clusterSize, len(maximumClique))
                totalCombination = cG.combinations()
                #  exhaust all possible combinations!!
                j = 0
                while j < totalCombination:
                    clusterList = []
                    clusterList = cG.next()
                    reOrder = 0
                    while reOrder < clusterSize:
                        # System.out.print(clusterList[reOrder]);
                        # add value each subclique in a maximum clique.  
                        compsub1.add(maximumClique.get(clusterList[reOrder]))
                        reOrder += 1
                    # System.out.print("\n");
                    # remove the repeated clusters existing in different cliques.
                    compsub1String = compsub1.__str__()
                    if compsub2.contains(compsub1String) == False:
                        compsub2.add(compsub1String)
                        # print("===============compsub1String :"+ compsub1String);
                        # judge the putative binding site.
                        judgeA(ocFile, compsub1, fileName, rFile, allAtomNoWater)
                    compsub1.clear()
                    j += 1
                clusterSize += 1
            i += 1
        #  end exhausting all pockets.		  
        #  Get elapsed time in milliseconds
        # elapsedTimeMillis2 = System.currentTimeMillis() - start
        # #  Get elapsed time in minutes
        # elapsedTimeMin2 = elapsedTimeMillis2 / (60 * 1000)
        # print(elapsedTimeMin2);

    # end predict ()
    def fmxlcA(self, ocFile, compsub, candidate, not_, fileName, oxygenNum):
        """ generated source for method fmxlcA """
        # don't need to change
        # 	% Abstract: 
        # 		    % Find all the maximal cliques of size >= 4, where C_m is the adjacency
        # 		    % matrix.  
        # 		    % Call function "judgeA" to evaluate the found maximal cliques.
        # 		% Version: 2.0
        # 		% Author: Kun Zhao
        # 		% Date: 2/1/2014
        # 		% Version: 1.2
        # 		% Author: Xue Wang
        # 		% Date: 7/25/2007
        # 		% Replace version 1.1
        # 		% Author: Hai Deng 
        # 		% Date: Sep., 2006
        # 		
        count_prd = 0
        # jj = 0
        # ii = 0
        # k = 0
        mFile = []
        matrixRow = [None] * len(ocFile)
        currDirectory = os.getcwd()
        for jj in range(len(not_)):
          bound = 1
          mFile = FileOperation.getEntriesAsList(currDirectory+"/outputdata/" +fileName + "_m.dat")
          notPlace = int(str(not_[jj]))
          indicationChar = str(mFile[notPlace]).split(" ")
          for k in range(len(mFile[:-1])):
            matrixRow[k] = int((indicationChar[k]))
          for k in range(len(candidate)):
            if matrixRow[int(str(candidate[k]))] == 0:							
                bound=0
                break
          if bound == 1:
            return count_prd    # branch and bound
        # condition of maximum clique
        if len(candidate) == 0:
          if len(compsub) >= 4:
            maxCliqueList = []
            maxCliqueList.append(compsub)
            maxClique[count_maxClique] = maxCliqueList
            count_maxClique +=1
        else:	
          for i in range(len(candidate)):    # loop through all items in candidate
            cand = str(candidate[i])
            compsub.append(cand)
            candidate1 = []
            not1 = []
            mFile = FileOperation.getEntriesAsList(currDirectory+"/outputdata/" +fileName + "_m.dat")
            mFile[:-1]
            candPlace = int(cand)
            indicationChar =str(mFile[candPlace]).split(" ");
            for k in range(len(mFile[:-1])):
              matrixRow[k] = int(indicationChar[k])
            
            for kk in range(len(candidate)):       # find the adjacency list of an oxygen in candidate
              candidatePlace = int(str(candidate[i+1]))
              if matrixRow[candidatePlace] == 1:
                candidate1.append(candidate[i+1])
            for j in range(len(not_)):
              if matrixRow[int(str(not_[j]))]==1:
                      not1.append(not_[j])
            
            # condition for search maximum clique of 
            self.fmxlcA(ocFile, compsub, candidate1, not1, fileName, oxygenNum) # recursive call 
                  

            if len(compsub) >= 1:
              compsub[-1]
            not_.append(cand)
        return count_prd

    def judgeA(self, ocFile, compsub, fileName, rFile, allAtomNoWater):
        """ generated source for method judgeA """
        # -----------------------------need to modify----------------------------------------//
        # % Filename: judgeA.c
        # 		%
        # 		% Abstract: 
        # 		    % Return count_prd (either 1 or 0) to indicate if a maximal clique is a putative Ca-binding site.
        # 		    % Modify tp_array to indicate if a clique is a prediction
        # 		%
        # 		% Version: 0.3
        # 		% Author: Kun Zhao
        # 		% Date: 06/06/2009
        # 		% Version: 0.2
        # 		% Author: Xue Wang
        # 		% Date: 1/25/2008
        # 		% Replace version 0.1
        # 		% Author: Xue Wang
        # 		% Date: Jun., 2007
        # 		
        count_prd = 0
        lengthLocat = 0
        angleVal = 0
        countCharge = 0
        flag = 0
        #  indicate that current clique contains putative binding site
        ii = int()
        kk = int()
        jj = 0
        x = [None] * 3
        #  intitialize calcium location.
        currDirectory = os.getcwd()
        while ii < len(compsub):
            if int(str((compsub[ii]))) < self.infor1:
                lengthLocat += 1
            ii += 1
        if lengthLocat < 4:
            flag = 2
        z = 0
        while z < len(compsub):
            ocLine = str(ocFile.get(Integer.parseInt(str(compsub.get(z)))))
            # System.out.print(ocLine.split(" +")[4] + " ");
            z += 1
        while ii < lengthLocat:
            ocLine = str(ocFile.get(Integer.parseInt(str(compsub.get(ii)))))
            resiName = (ocLine.split(" +"))[3]
            atomName = (ocLine.split(" +"))[2]
            if (resiName == "ASP" or resiName == "GLU") and atomName == "O" == False:
                countCharge = countCharge + 1
            if resiName == "ASN":
                countCharge = countCharge + 0.5
            ii += 1
        if flag == 0:
            conseResFil = ConsecutiveResiduesFilter()
            conseResFil.setParameters(ocFile, compsub, self.infor1)
            flag = conseResFil.getFlag()
            #  Distance and angle Filters 
            #  Find the max and min distance between an first shell carbon and the calcium center; return the coordinates of calcium center;
            # charge filter 
            if countCharge < 1 and flag == 0:
                flag = 1
            if countCharge >= 1 and flag == 0:
                p = [None] * 5
                #  carbon shell charge filter. 
                p = ChargeDistanceFilter.filter_distance(flag, ocFile, fileName, compsub, lengthLocat, self.carbonUpperLimit, self.carbonLowerLimit, self.infor1, self.gridInterval)
                x[0] = p[0]
                x[1] = p[1]
                x[2] = p[2]
                flag = int(p[4])
            # center of mass filter, carbon, mainchain oxygen, and atom clash filter.				
            if flag == 0:
                flag = CenterOfMassFilter.rFilter(x, ocFile, rFile, compsub, allAtomNoWater)
        #  Output the predicted ligand group and calcium location 
        if flag == 0:
            pdb.set_trace()         
            PutativeTrue.doPrint(ocFile, compsub, fileName, x)
        # System.out.print("\n");
        # System.out.print(flag);
        # System.out.print("\n");
# runmugc = MugControllCarbonShell()	
# runmugc.Predict('3cln')

### **ParsePDB**

In [19]:
class CenterOfMass(object):
    """ generated source for class CenterOfMass """
    @classmethod
    def getRCalpha(cls, args):
        """ generated source for method getRCalpha """
        fileList = []
        allAtom = []
        resultList = []
        atom = "allatom"
        carbonLine = ""
        oxygenLine = ""
        currDirectory = os.getcwd()
        fileList = FileOperation.getEntriesAsList(args)
        #  a list of PDBIDs
        i = 0
        while i < len(fileList)-1:
            #  iteratively process PDB files
            PDBID = (str(fileList[i]))[0:4]
            fileName = currDirectory + "/inputdata/" + PDBID + ".pdb"
            allAtom = ParsePDB.getLines(fileName, "ATOM", "Res", atom)
            FileOperation.saveResults(cls.simplifyModel(allAtom), currDirectory + "/outputdata/" + PDBID + "_COM.pdb", "w")
            # save results as file
            resultList = []
            i += 1

    @classmethod
    def simplifyModel(cls, allAtom):
        """ generated source for method simplifyModel """
        entries = []
        sideChain = []
        # len(allAtom)
        k = 0
        while k < len(allAtom):
            getString = str(allAtom[k])
            if ("CB" in getString[13:16]) != True:
                entries.append(getString)
                if ("CB" in getString[13:16]) == True and ("GLY" in getString[17:21]) == False:
                    sideChain.append(getString)
            else:
                h = k
                scResidue = ((str(allAtom[h]))[23:26])
                while h < len(allAtom) - 1 and (getString[23:26] == scResidue):
                    sideChain.append(str(allAtom[h]))
                    h += 1
                    scResidue = (str(allAtom[h]))[23:26]
                if h == len(allAtom) - 1 and (getString[23:26] == scResidue):
                    sideChain.append(str(allAtom[h]))
                entries.append(SideChainCenterOfMass.sideChainCalculation(sideChain))
                sideChain = []
                # skip to the next residue.
                if h < len(allAtom) - 1:
                    k = h - 1
                # deal with last residue
                if h == len(allAtom) - 1:
                    break
            k += 1
        return entries


In [20]:
class CombineFiles(object):
    """ generated source for class CombineFiles """
    @classmethod
    def combineSurfix(cls, args, fileType):
      """ generated source for method combineSurfix """
      inputLine = None
      filePrefixes = []
      inputLine = fileType
      filePrefixes = inputLine.split(" ")
      fileList = []
      fileList = FileOperation.getEntriesAsList(args)
      #  a list of PDBIDs
      currDirectory = os.getcwd()
      finalResult = []
      i = 0
      while i < len(fileList)-1:
        PDBID = (str(fileList[i]))[0:4]
        outFileName = currDirectory + "/outputdata/" + PDBID + "_out.dat"
        inputList = []
        j = 0
        while j < len(filePrefixes):
          print('Combinesurfix length', len(filePrefixes))
          inputList.append(currDirectory + "/outputdata/" + PDBID + "_" + filePrefixes[j] + ".dat")
          print('Current inputlist: ', inputList)
          j += 1
        # FileOperation.concatenateFiles(outFileName, inputList)
        shutil.copy(inputList[0], outFileName)
        i += 1
      i = 0
      while i < len(fileList)-1:
        PDBID = (str(fileList[i]))[0:4]
        outFileName = currDirectory + "/outputdata/" + PDBID + "_out.dat"
        inputFile = []
        inputFile = FileOperation.getEntriesAsList(currDirectory + "/outputdata/" + PDBID + "_out.dat")
        j = 0
        while j < len(inputFile):
            line = str((inputFile[j]))
            sub = line[14:15]
            if sub == "N" or sub == "S":
                line = line + "   0.000    0.000    0.000"
            sub2 = line[0:6]
            if sub2 == "HETATM":
                line = line + "   0.000    0.000    0.000"
            finalResult.append(line)
            # print('Final result is: ',finalResult)
            j += 1
        FileOperation.saveResults(finalResult, currDirectory + "/outputdata/" + PDBID + "_f1.dat", "w")
        finalResult = []
        file_ = outFileName
        os.remove(file_)
        i += 1


In [21]:
class MetalBindingCC(object):
    """ generated source for class MetalBindingCC """
    @classmethod
    def getCC(cls, args):
        """ generated source for method getCC """
        fileList = []
        OAndCList = []
        resultList = []
        atom = "O"
        bondedAtom = "C"
        #  Bonded by "O";
        firstShellCarbonLine = ""
        secondShellCarbonLine = ""
        oxygenLine = ""
        currDirectory = os.getcwd()
        fileList = FileOperation.getEntriesAsList(args)
        #  a list of PDBIDs
        i = 0
        while i < len(fileList)-1:
            #  iteratively process PDB files
            PDBID = (str(fileList[i]))[0:4]
            fileName = currDirectory + "/inputdata/" + PDBID + ".pdb"
            OAndCList = ParsePDB.getBondedLines(fileName, atom, bondedAtom)
            # initialize secondShellCarbonLine
            # firstShellCarbonLine = ((str(OAndCList[0]))[30:38] + " " + (str(OAndCList[0]))[38:46] + " " + (str(OAndCList[0]))[46:54])
            # print("firstShellCarbonLine: ",firstShellCarbonLine)
            # j = 1
            # while j < len(OAndCList):
            for j ,(OAndC) in enumerate(OAndCList):
              # #  process lines in one PDB file
              firstShellCarbonLine = ((str(OAndC))[30:38] + " " + (str(OAndC))[38:46] + " " + (str(OAndC))[46:54])
              # getString = str(OAndCList[j])
              # # print(getString)
              # if ("ATOM" in getString[0:4]) and (atom == getString[13]) and (("A" == getString[21]) or (getString[16].isspace()) and ("2" == getString[10]) != True): 
              #   oxygenLine = getString[0:6] + " " + getString[6:11] + " " + getString[12:16] + " " + " " + " " + getString[17:20] + " " + " " + " " + getString[22:26] + " " + " " + " "
              #   # print('For oxygen line, fist, second and carbon shells: ',oxygenLine + firstShellCarbonLine + " " + secondShellCarbonLine + "   " + getString[21:22])
              #   resultList.append(oxygenLine + firstShellCarbonLine + " " + secondShellCarbonLine + "   " + getString[21:22])
              # if (("ATOM" in getString[0:4]) and "CB" in getString[13:15]) and ((("A" in getString[21]) or getString[16].isspace())):
              #   secondShellCarbonLine = ((str(OAndCList[j - 3]))[30:38] + " " + (str(OAndCList[j - 3]))[38:46] + " " + (str(OAndCList[j - 3]))[46: 54])
              #   print('Current Secondcarbon shell',secondShellCarbonLine)
              # else:
              #   secondShellCarbonLine = firstShellCarbonLine
              # if ("ATOM" in getString[0:4]) and (bondedAtom in getString[13]) and (("A" in getString[16]) or getString[16].isspace()):
              #     firstShellCarbonLine = getString[30:38] + " " + getString[38:46] + " " + getString[46:54]
              if ("ATOM" in str(OAndC)[0:4]) and (atom == str(OAndC)[13]) and (("A" == str(OAndC)[21]) or (str(OAndC)[16].isspace()) and ("2" == str(OAndC)[10]) != True): 
                oxygenLine = str(OAndC)[0:6] + " " + str(OAndC)[6:11] + " " + str(OAndC)[12:16] + " " + " " + " " + str(OAndC)[17:20] + " " + " " + " " + str(OAndC)[22:26] + " " + " " + " "
                # print('For oxygen line, fist, second and carbon shells: ',oxygenLine + firstShellCarbonLine + " " + secondShellCarbonLine + "   " + getString[21:22])
                resultList.append(oxygenLine + firstShellCarbonLine + " " + secondShellCarbonLine + "   " + str(OAndC)[21:22])
              if (("ATOM" in str(OAndC)[0:4]) and "CB" in str(OAndC)[13:15]) and ((("A" in str(OAndC)[21]) or str(OAndC)[16].isspace())):
                secondShellCarbonLine = ((str(OAndCList[j - 3]))[30:38] + " " + (str(OAndCList[j - 3]))[38:46] + " " + (str(OAndCList[j - 3]))[46: 54])
                # print('Current Secondcarbon shell',secondShellCarbonLine)
              else:
                secondShellCarbonLine = firstShellCarbonLine
              if ("ATOM" in str(OAndC)[0:4]) and (bondedAtom in str(OAndC)[13]) and (("A" in str(OAndC)[16]) or str(OAndC)[16].isspace()):
                  firstShellCarbonLine = str(OAndC)[30:38] + " " + str(OAndC)[38:46] + " " + str(OAndC)[46:54]
              # j += 1
            # print(resultList)
            FileOperation.saveResults(resultList, currDirectory + "/outputdata/" + PDBID + "_cc.dat", "w")
            # save results as file
            resultList = []
            i += 1


In [22]:
class MetalBindingM(object):
    """ generated source for class MetalBindingM """
    @classmethod
    def getAdjacencyMatrix(cls, args, cutOff):
        """ generated source for method getAdjacencyMatrix """
        threshold = cutOff
        fileList = []
        oxyList = []
        oxyLine = ""
        sizeOxyList = int()
        x = 0.0
        y = 0.0
        z = 0.0
        dist = Dist()
        currDirectory = os.getcwd()
        fileList = FileOperation.getEntriesAsList(args)
        #  a list of PDBIDs   
        i = 0
        while i < len(fileList)-1:
            #  iteratively process PDB files
            countATOM = 0
            countHETATM = 0
            PDBID = (str(fileList[i]))[0:4]
            fileName = currDirectory + "/outputdata/" + PDBID + "_f1.dat"
            oxyList = FileOperation.getEntriesAsList(fileName)
            sizeOxyList = len(oxyList)-1
            print('sizeOxyList: ',sizeOxyList)
            h, w = sizeOxyList, 3;
            coordMatrix = [[0 for x in range(w)] for y in range(h)]
            indicationM = 0
            j = 0
            while j < sizeOxyList-1:
                #  get coordinate for each oxy atom
                oxyLine = str(oxyList[j])
                # print('This is the oxyLine: ', oxyList[j])
                try:
                    x = float(oxyLine[33:42])
                    y = float(oxyLine[42:51])
                    z = float(oxyLine[51:59])
                except Exception as e:
                    print("Error! in oxyLine")
                coordMatrix[j][0] = x
                coordMatrix[j][1] = y
                coordMatrix[j][2] = z
                if (oxyLine.startswith("ATOM") or oxyLine.startswith("HETATM")) and ("HOH" in oxyLine[20:23]) == False:
                    countATOM = countATOM + 1
                if oxyLine.startswith("HETATM") and (oxyLine[20:23]=="HOH") == True:
                    countHETATM = countHETATM + 1
                j += 1
            bw = open(currDirectory + "/outputdata/" + PDBID + "_m.dat", "wb")
            bw2 = open(currDirectory + "/outputdata/" + PDBID + "_info1.dat", "wb")
            try:
              j = 0
              while j < sizeOxyList:
                  k = 0
                  while k < sizeOxyList:
                      distance = Dist.distThreeD(coordMatrix[j][0], coordMatrix[j][1], coordMatrix[j][2], coordMatrix[k][0], coordMatrix[k][1], coordMatrix[k][2])
                      # print(coordMatrix[j][0], coordMatrix[j][1], coordMatrix[j][2], coordMatrix[k][0], coordMatrix[k][1], coordMatrix[k][2])
                      # print('The distance is: ',distance,'and the threshold is: ' ,threshold)
                      if distance < threshold:
                          indicationM = 1
                      else:
                          indicationM = 0
                      bw.write(str(indicationM))
                      bw.write(" ")
                      k += 1
                  bw.write("\n")
                  j += 1
              bw2.write(str(countATOM )+ " " + str(countHETATM) + "\n")
              bw.close()
              bw2.close()
            except IOError as e:
                print("Error! 2 in MBM")
            i += 1


# Run **Demo**

## **Main**

In [23]:
%%bash setupmugcpytest.sh
ls inputdata


3cln
Created all needed folders


--2019-09-11 15:59:44--  https://files.rcsb.org/download/3cln.pdb
Resolving files.rcsb.org (files.rcsb.org)... 128.6.244.12
Connecting to files.rcsb.org (files.rcsb.org)|128.6.244.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘3cln.pdb’

     0K .......... .......... .......... .......... .......... 1.33M
    50K .......... .......... .......... .......... .......... 1.48M
   100K .......... .......... .......... ......                14.5M=0.07s

2019-09-11 15:59:45 (1.84 MB/s) - ‘3cln.pdb’ saved [139320]



In [24]:
# for i ,(name) in enumerate(ppdb.df['OTHERS']['record_name']):
#   print(i, name)

In [25]:
class MUGC(object):
    """ generated source for class MUGC """
    @classmethod
    def main(cls, args):
        """ generated source for method main """
        currDirectory = os.getcwd()
        fileName = os.path.join(currDirectory, "inputdata", "list.txt")
        print(fileName)
        NMRFileName = ""
        argsMugC = [None] * 4
        PDBID = []
        PDBIDList = []
        NMRModels = []
        PDBID = FileOperation.getEntriesAsList(fileName)
        # set default parameters
        # GridInterval for moving the calcium center at the most suitable positions.  Parameter -i
        gridInterval = "4"
        # The lower limit of distance between carbon atoms and calcium center, e.g. observed from 2BBM.  Parameter -l
        carbonLowerLimit = "1.74"
        # The upper limit of distance between carbon atoms and calcium center.  Parameter -u
        carbonUpperLimit = "4.9"
        #  read parameters from user input.	
        # if len(args):
        #     if Arrays.asList(args).indexOf("-i") != -1:
        #         print(args[Arrays.asList(args).indexOf("-i") + 1])
        #     if (Arrays.asList(args).indexOf("-l")) != -1:
        #         print(args[Arrays.asList(args).indexOf("-l") + 1])
        #     if Arrays.asList(args).indexOf("-u") != -1:
        #         print(args[Arrays.asList(args).indexOf("-u") + 1])
        #     if Arrays.asList(args).indexOf("-help") != -1:
        #         print("A Proper Usage example: java ./MugC -i 4 -l 1.74 -u 4.9")
        #         print("-i GridInterval for moving the calcium center at the most suitable positions. Default=4")
        #         print("-l The lower limit of distance between carbon atoms and calcium center. Default=1.74")
        #         print("-u The upper limit of distance between carbon atoms and calcium center. Default=4.9")
        #         System.exit(0)
        argsMugC[1] = gridInterval
        argsMugC[2] = carbonLowerLimit
        argsMugC[3] = carbonUpperLimit
        # Save, seperate and manipulate NMR structures. Return the total number of model in the NMR. 
        NMRModels = NMRSeperate.nMRSeperate(fileName)
        j = 0
        while j < len(PDBID):
            # in case that only one NMR model
            if not NMRModels:
                NMRFileName = currDirectory + "/inputdata/list.txt"
            else:
                NMRFileName = currDirectory + "/inputdata/" + PDBID[j] + "list.txt"
            PDBIDList = FileOperation.getEntriesAsList(NMRFileName)
            #  Preparing the intermediate data (including the topology graph) for further processing 
            #  extract Calcium from PDB file, if there is one.
            MetalBindingMetal.getMetal(NMRFileName, "CA")
            #  extract the carbon atoms for the Carbon Shell.
            MetalBindingCC.getCC(NMRFileName)
            #  genrate file from cc file, could be change to cc file;  
            CombineFiles.combineSurfix(NMRFileName, "cc")
            #  calculate the topological graph based on adjacent matrix.
            MetalBindingM.getAdjacencyMatrix(NMRFileName, 8.3)
            #  calculate center of mass coordinates of each side-chain.
            CenterOfMass.getRCalpha(NMRFileName)
            i = 0
            while i < len(PDBIDList)-1:
                argsMugC[0] = str((PDBIDList[i]))
                runmugc = MugControllCarbonShell()
                print(argsMugC[0])		
                runmugc.Predict(argsMugC)
                i += 1
            # print("PDBIDList size:" +len(PDBIDList));
            # generate PDB prediction file.
            NMRSeperate.nMRmergePDB(PDBIDList, str(PDBID[j]))
            j += 1
        print("Completed!")


if __name__ == '__main__':
    import sys
    MUGC.main(sys.argv)


/Users/mashiku/Documents/DevTest/inputdata/list.txt
File: /Users/mashiku/Documents/DevTest/outputdata/3cln_ca.dat just written
Combinesurfix length 1
Current inputlist:  ['/Users/mashiku/Documents/DevTest/outputdata/3cln_cc.dat']
sizeOxyList:  0


TypeError: a bytes-like object is required, not 'str'

In [ ]:
f = open("/content/predictionResults/results.dat", "r")

print(f.read())